In [1]:
%%capture
!pip install torch transformers datasets

## Import - model - inference

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')
input_ids = tokenizer.encode("summarize: Hello, my dog is cute", return_tensors="pt")  
outputs = model.generate(input_ids)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
decoded_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
decoded_text

'my dog is cute and cute.'

In [46]:
def translate_en_de(en_sentence,device="cuda"):
    input_ids = tokenizer.encode(en_sentence, return_tensors="pt").to(device) 
    outputs = model.to(device).generate(input_ids)
    decoded_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded_text

In [47]:
translate_en_de("summarize hi")

'hi hi, i f.'

## Test print loss with model

In [5]:
english_sentence = "hello I'm from Vietnam"
germany_sentence = "Hallo, ich komme aus Vietnam"
input_ids_en = tokenizer.encode(english_sentence, return_tensors="pt")
input_ids_ge = tokenizer.encode(germany_sentence, return_tensors="pt")

In [6]:
# model(input_ids = input_ids_en, labels = input_ids_en)

## Create dataset English - Germany

In [7]:
from datasets import load_dataset

# Load the WMT16 German-to-English dataset
dataset = load_dataset("wmt16", "de-en")

README.md:   0%|          | 0.00/11.1k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/282M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/267M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/277M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/343k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/475k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

In [10]:
# Check available splits
print(dataset)  # Shows train, validation, and test splits

# Get a sample from the training set
sample = dataset["train"][0]
print("German:", sample["translation"]["de"])
print("English:", sample["translation"]["en"])


DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 100
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 100
    })
})
German: Wiederaufnahme der Sitzungsperiode
English: Resumption of the session


In [11]:
dataset['train'] = dataset['train'].select(range(100))
dataset['test'] = dataset['test'].select(range(100))
dataset['validation'] = dataset['validation'].select(range(100))
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 100
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 100
    })
})

In [12]:
from datasets import DatasetDict, Dataset
import pandas as pd
# Function to transform dataset into two-column format
def transform_to_two_columns(dataset_dict):
    new_datasets = {}

    for split in dataset_dict.keys():  # Iterate over train, validation, test
        # Convert to DataFrame
        df = pd.DataFrame(dataset_dict[split]["translation"])
        
        # Convert back to Hugging Face Dataset
        new_datasets[split] = Dataset.from_pandas(df)

    return DatasetDict(new_datasets)

In [13]:
new_dataset = transform_to_two_columns(dataset)
new_dataset

DatasetDict({
    train: Dataset({
        features: ['de', 'en'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['de', 'en'],
        num_rows: 100
    })
    test: Dataset({
        features: ['de', 'en'],
        num_rows: 100
    })
})

In [14]:
new_dataset['train'][0]

{'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}

## Tokenizer

In [25]:
# Tokenization function
def tokenize_function(examples):
    model_inputs = tokenizer(examples["en"], padding="max_length", truncation=True, max_length=100)
    labels = tokenizer(examples["de"], padding="max_length", truncation=True, max_length=100)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [26]:
# Apply tokenization
tokenized_datasets = new_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [27]:
tokenized_datasets = tokenized_datasets.remove_columns(["de"])
tokenized_datasets = tokenized_datasets.remove_columns(["en"])
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

## Train

In [28]:
from torch.utils.data import DataLoader
tokenized_datasets.set_format("torch")

In [55]:
# Create dataloaders
train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=8, shuffle=True)
test_dataloader = DataLoader(tokenized_datasets["test"], batch_size=8, shuffle=False)

In [56]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

In [57]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

epochs = 100  # Number of training epochs

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} - Loss: {total_loss / len(train_dataloader)}")


cuda
Epoch 1 - Loss: 0.5116732716560364
Epoch 2 - Loss: 0.48630619507569534
Epoch 3 - Loss: 0.45488625306349534
Epoch 4 - Loss: 0.42544485284731937
Epoch 5 - Loss: 0.45508502423763275
Epoch 6 - Loss: 0.4035171579856139
Epoch 7 - Loss: 0.4024740629471265
Epoch 8 - Loss: 0.37912344474058884
Epoch 9 - Loss: 0.36916810503372777
Epoch 10 - Loss: 0.3674954898082293
Epoch 11 - Loss: 0.35202881693840027
Epoch 12 - Loss: 0.3463290322285432
Epoch 13 - Loss: 0.3244329954569156
Epoch 14 - Loss: 0.32662343978881836
Epoch 15 - Loss: 0.3373142182826996
Epoch 16 - Loss: 0.30261998910170335
Epoch 17 - Loss: 0.30577193200588226
Epoch 18 - Loss: 0.30290413819826567
Epoch 19 - Loss: 0.28567400918557095
Epoch 20 - Loss: 0.2877955310619794
Epoch 21 - Loss: 0.2814867989375041
Epoch 22 - Loss: 0.2692330754720248
Epoch 23 - Loss: 0.2628253618111977
Epoch 24 - Loss: 0.27079426554533154
Epoch 25 - Loss: 0.25246513233735013
Epoch 26 - Loss: 0.24969843946970427
Epoch 27 - Loss: 0.24450019747018814
Epoch 28 - Loss:

In [58]:
new_dataset['train'][0]

{'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}

In [59]:
translate_en_de(new_dataset['train'][0]["de"])

'Wiederaufnahme der Sitzungsperiode Wahlperiode in der Sitzungsperio'

In [60]:
translate_en_de(new_dataset['train'][0]["en"])


'Wiederaufnahme der Sitzungsperiode'